In [2]:
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from IPython.display import display
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.tree import DecisionTreeRegressor
# from sklearn.externals import joblib
from sklearn import preprocessing
import statsmodels as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.formula.api as formula
from statsmodels.tsa.seasonal import seasonal_decompose
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
warnings.filterwarnings(action='ignore')


In [3]:
file_path = '../data/raw/data-train/input/S0000137-Ngoc Khanh.csv'

In [34]:
df = pd.read_csv(file_path)

In [35]:
df = df.iloc[:, 1:]

In [7]:
df

,timestamp,PM2.5,humidity,temperature
0,27/05/2020 12:00,23.66,64.09,29.96
1,27/05/2020 13:00,23.08,63.05,30.07
2,27/05/2020 14:00,25.68,67.51,29.77
3,27/05/2020 15:00,26.37,71.17,29.11
4,27/05/2020 16:00,27.57,71.60,29.08
...,...,...,...,...
8995,06/06/2021 07:00,NaN,NaN,NaN
8996,06/06/2021 08:00,NaN,NaN,NaN
8997,06/06/2021 09:00,NaN,NaN,NaN
8998,06/06/2021 10:00,NaN,NaN,NaN


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9000 entries, 0 to 8999
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   timestamp    9000 non-null   object 
 1   PM2.5        8103 non-null   float64
 2   humidity     8319 non-null   float64
 3   temperature  8319 non-null   float64
dtypes: float64(3), object(1)
memory usage: 281.4+ KB


In [36]:
# check for duplicated values and null values
print('Are there any duplicated values in our data ? : {}\n'.format(
    df.duplicated().any()))
print('The total number of null values in each colum:')
display(df.isnull().sum())


Are there any duplicated values in our data ? : False

The total number of null values in each colum:


timestamp        0
PM2.5          897
humidity       681
temperature    681
dtype: int64

### Handle NaN values

In [37]:
# fill in the missing values with the mean of the particular column
df['PM2.5'].fillna(value = df['PM2.5'].mean(), inplace = True)
df['humidity'].fillna(value=df['humidity'].mean(), inplace=True)
df['temperature'].fillna(value=df['temperature'].mean(), inplace=True)


In [19]:
# let's check the data again if there are any missing values
df.isnull().any()


timestamp      False
PM2.5          False
humidity       False
temperature    False
dtype: bool

### Scale

In [39]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler


In [40]:
col_names = ['humidity', 'temperature']


In [41]:
ct = ColumnTransformer([
    ('Scaler', StandardScaler(), col_names)
], remainder='passthrough')


In [44]:
df[col_names] = ct.fit_transform(df[col_names])

In [ ]:
df.to_csv('../data/processed/data-train/input/S0000137-Ngoc Khanh.csv')


### Process Data

In [ ]:
!python src/data/data_processing.py